# Autograd

autograd방식으로 backpropagation을 이용한 파라미터 업데이트를 쉽게 구현할 수 있다.
파이토치에는 torch.autograd라 불리는 자동 미분 엔진이 내장되어 있다. 모든 계산 그래프에 대한 변화도의 자동 계산을 지원한다.

In [1]:
import torch

if torch.cuda.is_available():
  DEVICE = torch.device('cuda')
else:
  DEVICE = torch.device('cpu')

In [2]:
# 배치 사이즈
BATCH_SIZE = 64
# 인풋데이터의 사이즈 겸 입력층의 노드 수
INPUT_SIZE = 1000
# 즉, (64, 1000) 데이터가 쓰인다.

HIDDEN_SIZE = 100
OUTPUT_SIZE = 10

In [10]:
# requires_grad의 값은 텐서를 생성할 때나, 나중에 `x.requires_grad_(True)` 메소드를 이용해 설정 가능

In [3]:
x = torch.randn(BATCH_SIZE, INPUT_SIZE, device=DEVICE, dtype=torch.float, requires_grad=False)
y = torch.randn(BATCH_SIZE, OUTPUT_SIZE, device=DEVICE, dtype=torch.float, requires_grad=False)
# x,y는 입력과 출력값이므로 gradient를 구할 필요가 없어서 requires_grad를 False로 둔다.

# w1, w2는 gradient를 구해서 값을 업데이트 해줘야하기 때문에 requires_grad를 True로 둔다.
w1 = torch.randn(INPUT_SIZE, HIDDEN_SIZE, device=DEVICE, dtype=torch.float, requires_grad=True)
w2 = torch.randn(HIDDEN_SIZE, OUTPUT_SIZE, device=DEVICE, dtype=torch.float, requires_grad=True)

In [6]:
print(x, '\n', x.shape)

tensor([[-0.1393,  1.2510,  0.0170,  ...,  1.7679,  0.1987, -0.6384],
        [ 0.0098, -0.3368,  1.9453,  ..., -0.4321,  1.8328, -0.6544],
        [ 0.5819,  1.5054,  0.0996,  ..., -0.1860, -0.7522,  0.0107],
        ...,
        [ 0.6359,  0.5900,  1.9819,  ...,  1.0665,  1.2601, -0.9228],
        [ 0.4386, -1.8881,  1.1685,  ...,  0.9623,  0.2759, -1.0693],
        [ 1.6388, -0.4228, -0.5406,  ...,  1.6179, -0.0967, -0.4070]],
       device='cuda:0') 
 torch.Size([64, 1000])


In [8]:
# print(y, '\n', y.shape)

In [22]:
LEARNING_RATE = 1e-6
cnt = 0
for t in range(1, 501):
  y_pred = x.mm(w1).clamp(min=0).mm(w2)
  
  loss = (y_pred - y).pow(2).sum()
  if t % 100 == 0:
    print('Iteration : ', t, '\t', 'Loss : ', loss.item())
  # loss 값에 backward() 메서드를 이용하면 각 파라미터 값에 대해 gradient를 계산하고 이를 통해 backpropagation을 한다는 것을 의미한다.
  loss.backward()
  if cnt < 2:
    print('w1.grad = ', w1.grad)
    print('w2.grad = ', w2.grad)
  else :
    pass
  cnt+=1
  # 변화도 추적을 멈춰야 하는 이유
  # 1. 신경망의 일부 매개변수를 고정된 매개변수(frozen parameter)로 표시. 사전 학습된 신경망을 미세조정할 때
  # 매우 일반적인 시나리오
  # 2. 변화도를 추적하지 않는 텐서의 연산이 더 효율적이기 때문에, 순전파 단계만 수행할 때 연산속도가 향상된다.
  with torch.no_grad():
    w1 -= LEARNING_RATE * w1.grad
    w2 -= LEARNING_RATE * w2.grad

    # w1.grad, w2.grad를 0으로 초기화 해주는 것. 안해주면 계속 값이 누적되어 원하는 미분값이 아니게된다.
    w1.grad.zero_()
    w2.grad.zero_()

w1.grad =  tensor([[-1.3685e-04,  2.9731e-04, -7.1525e-04,  ..., -8.0449e-04,
         -1.1059e-03, -6.6028e-04],
        [-1.4370e-03,  1.0292e-03,  3.5090e-04,  ...,  2.5063e-03,
         -1.4701e-03, -7.9356e-04],
        [ 6.1893e-04, -7.6096e-04, -5.4062e-05,  ...,  4.1706e-04,
          2.5333e-05, -2.0589e-04],
        ...,
        [-2.8334e-03, -1.3185e-03,  1.5759e-03,  ...,  9.5276e-05,
         -8.9593e-04,  4.7857e-04],
        [ 2.3867e-03,  1.2191e-03,  6.3393e-04,  ...,  1.1159e-03,
          5.5712e-04,  3.9353e-04],
        [ 9.1751e-04,  1.1608e-03,  6.8047e-04,  ...,  1.7243e-03,
         -1.3550e-03, -7.2401e-04]], device='cuda:0')
w2.grad =  tensor([[ 2.2689e-03,  1.0447e-02, -2.4862e-04, -1.7026e-03,  4.5631e-03,
         -5.1476e-03,  2.3811e-03,  1.2385e-02,  6.4466e-03, -2.1852e-03],
        [-8.2926e-03,  6.0621e-04, -2.0359e-03, -1.8932e-03,  4.1475e-03,
          6.8679e-03,  5.0907e-03, -1.9312e-03, -1.0983e-02,  1.8484e-03],
        [-6.5578e-03, -1.6015e-

In [17]:
print(w1.grad)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')


In [18]:
print(w1.grad.shape)

torch.Size([1000, 100])
